# Processing Big Data - Deequ Analysis

© Explore Data Science Academy

## Honour Code
I **KUSASALETHU**, **SITHOLE**, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the [EDSA honour code](https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).
    Non-compliance with the honour code constitutes a material breach of contract.


## Context

Having completed manual data quality checks, it should be obvious that the process can become quite cumbersome. As the Data Engineer in the team, you have researched some tools that could potentially save the team from having to do this cumbersome work. In your research, you have come a across a tool called [Deequ](https://github.com/awslabs/deequ), which is a library for measuring the data quality of large datasets.

<div align="center" style="width: 600px; font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://github.com/Explore-AI/Pictures/raw/master/data_engineering/transform/predict/DataQuality.jpg"
     alt="Data Quality"
     style="float: center; padding-bottom=0.5em"
     width=100%/>
     <p><em>Figure 1. Six dimensions of data quality</em></p>
</div>

You present this tool to your manager; he is quite impressed and gives you the go-ahead to use this in your implementation. You are now required to perform some data quality tests using this automated data testing tool.
 

> ## 🚩️ Important Notice 🚩️
>
>To successfully run `pydeequ` without any errors, please make sure that you have an environment that is running pyspark version 3.0.
> You are advised to **create a new conda environment** and install this specific version of pyspark to avoid any technical issues:
>
> `pip install pyspark==3.0`

<br>

## Import dependencies

If you do not have `pydeequ` already installed, install it using the following command:
- `pip install pydeequ`

In [204]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pydeequ
from pydeequ.analyzers import *
from pydeequ.profiles import *
from pydeequ.suggestions import *
from pydeequ.checks import *
from pydeequ.verification import *

from pyspark.sql import SparkSession, Row
from pyspark.sql import functions as F
from pyspark.sql.types import DecimalType, DoubleType, IntegerType, DateType, NumericType, StructType, StringType, StructField, LongType

In [205]:
spark = (SparkSession
    .builder
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .getOrCreate())

## Read data into spark dataframe

In this notebook, we set out to run some data quality tests, with the possiblity of running end to end on the years 1963, 1974, 1985, 1996, 2007, and 2018. 

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Data_ingestion_student_version.ipynb` notebook to create the parquet files for the following years:
>       - 1963
>       - 1974
>       - 1985
>       - 1996
>       - 2007
>       - 2018
>
>2. Ingest the data for the for the years given above. You should only do it one year at a time.
>3. Ingest the metadata file.


When developing your code, it will be sufficient to focus on a single year. However, after your development is done, you will need to run this notebook for all of the given years above so that you can answer all the questions given in the Data Testing MCQ.

In [206]:
#Ingest stock data of year
# Use this variable (year) to determine which year your are focusing on
year = 1963

df = spark.read.parquet(f"data/processed_data/{year}/ingested_data")
df.show(5)


+-------------------+----------+----------+---------+----------+-----------+--------+-----+
|               date|      open|      high|      low|     close|  adj_close|  volume|stock|
+-------------------+----------+----------+---------+----------+-----------+--------+-----+
|1963-10-28 00:00:00|  6.924645|  6.940665| 6.908625|  6.908625|  1.6844347| 39900.0|   AA|
|1963-10-28 00:00:00|  6.488943| 6.5123687| 6.465517|  6.465517|  1.5472991| 42600.0| ARNC|
|1963-10-28 00:00:00|0.56790125|0.56995887|0.5617284|0.56995887| 0.11004562|789600.0|   BA|
|1963-10-28 00:00:00| 1.8541666| 1.9166666|1.8541666| 1.9166666|  0.1730804|319200.0|  CAT|
|1963-10-28 00:00:00|       0.0| 4.0859375|4.0234375|  4.046875|0.067614794|156800.0|  CVX|
+-------------------+----------+----------+---------+----------+-----------+--------+-----+
only showing top 5 rows



In [215]:
#Ingest metadata of stock data
metadata_df = spark.read.csv(r"data\symbols_valid_meta.csv", header=True, inferSchema=True)

metadata_df.show(5)

+-------------+------+--------------------+----------------+---------------+---+--------------+----------+----------------+----------+-------------+----------+
|Nasdaq Traded|Symbol|       Security Name|Listing Exchange|Market Category|ETF|Round Lot Size|Test Issue|Financial Status|CQS Symbol|NASDAQ Symbol|NextShares|
+-------------+------+--------------------+----------------+---------------+---+--------------+----------+----------------+----------+-------------+----------+
|            Y|     A|Agilent Technolog...|               N|               |  N|         100.0|         N|            NULL|         A|            A|         N|
|            Y|    AA|Alcoa Corporation...|               N|               |  N|         100.0|         N|            NULL|        AA|           AA|         N|
|            Y|  AAAU|Perth Mint Physic...|               P|               |  Y|         100.0|         N|            NULL|      AAAU|         AAAU|         N|
|            Y|  AACG|ATA Creativity Gl.

## **Run tests on the dataset**

## Test 1 - Null values ⛔️
For the first test, you are required to check the data for completeness.

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to check for missing values in the data. 
>2. Display the results of your test.
>
> *You may use as many cells as necessary*


In [208]:
#Nulls Check (Completeness)
# Create checks for completeness of each column
check = Check(spark, CheckLevel.Error, "Completeness Check") \
    .hasCompleteness("date", lambda x: x ==1.0) \
    .hasCompleteness("open", lambda x: x ==1.0) \
    .hasCompleteness("high", lambda x: x ==1.0) \
    .hasCompleteness("low", lambda x: x ==1.0) \
    .hasCompleteness("close", lambda x: x ==1.0) \
    .hasCompleteness("adj_close", lambda x: x ==1.0) \
    .hasCompleteness("volume", lambda x: x ==1.0) \
    .hasCompleteness("stock", lambda x: x ==1.0)

# Run the verification suite
verification_result = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(check) \
    .run()

# Convert the result to a DataFrame
result_df = VerificationResult.checkResultsAsDataFrame(spark, verification_result)
result_df.show(truncate=False)


+------------------+-----------+------------+---------------------------------------------------------+-----------------+------------------+
|check             |check_level|check_status|constraint                                               |constraint_status|constraint_message|
+------------------+-----------+------------+---------------------------------------------------------+-----------------+------------------+
|Completeness Check|Error      |Success     |CompletenessConstraint(Completeness(date,None,None))     |Success          |                  |
|Completeness Check|Error      |Success     |CompletenessConstraint(Completeness(open,None,None))     |Success          |                  |
|Completeness Check|Error      |Success     |CompletenessConstraint(Completeness(high,None,None))     |Success          |                  |
|Completeness Check|Error      |Success     |CompletenessConstraint(Completeness(low,None,None))      |Success          |                  |
|Completeness

## Test 2 - Zero Values 🅾️

For the second test, you are required to check for zero values within the dataset.

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to check for zero values within the data. 
>2. Display the results of your test.
>
> *You may use as many cells as necessary*

In [209]:
#Zeros Check (Validity)
# # Create checks for zeros in each column
check = Check(spark, CheckLevel.Error, "Zeros Check") \
    .satisfies("open != 0", "zero_open") \
    .satisfies("high != 0", "zero_high") \
    .satisfies("low != 0", "zero_low") \
    .satisfies("close != 0", "zero_close") \
    .satisfies("adj_close != 0", "zero_adj_close") \
    .satisfies("volume != 0", "zero_volume")

# Run the verification suite
verification_result = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(check) \
    .run()

# Convert the result to a DataFrame
result_df = VerificationResult.checkResultsAsDataFrame(spark, verification_result)
result_df.show(truncate=False)

+-----------+-----------+------------+--------------------------------------------------------------------------------+-----------------+------------------------------------------------------------------+
|check      |check_level|check_status|constraint                                                                      |constraint_status|constraint_message                                                |
+-----------+-----------+------------+--------------------------------------------------------------------------------+-----------------+------------------------------------------------------------------+
|Zeros Check|Error      |Error       |ComplianceConstraint(Compliance(zero_open,open != 0,None,List(),None))          |Failure          |Value: 0.450199203187251 does not meet the constraint requirement!|
|Zeros Check|Error      |Error       |ComplianceConstraint(Compliance(zero_high,high != 0,None,List(),None))          |Success          |                                           

## Test 3 - Negative values ➖️
The third test requires you to check that all values in the data are positive.

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to check negative values within the dataset. 
>2. Display the results of your test.
>
> *You may use as many cells as necessary*

In [210]:
#Negative Values Check (Validity)
# # Create checks for negatives in each column
check = Check(spark, CheckLevel.Error, "Negative Check") \
    .satisfies("open >= 0", "negative_open") \
    .satisfies("high >= 0", "negative_high") \
    .satisfies("low >= 0", "negative_low") \
    .satisfies("close >= 0", "negative_close") \
    .satisfies("adj_close >= 0", "negative_adj_close") \
    .satisfies("volume >= 0", "negative_volume")

# Run the verification suite
verification_result = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(check) \
    .run()

# Convert the result to a DataFrame
result_df = VerificationResult.checkResultsAsDataFrame(spark, verification_result)
result_df.show(truncate=False)

+--------------+-----------+------------+------------------------------------------------------------------------------------+-----------------+------------------+
|check         |check_level|check_status|constraint                                                                          |constraint_status|constraint_message|
+--------------+-----------+------------+------------------------------------------------------------------------------------+-----------------+------------------+
|Negative Check|Error      |Success     |ComplianceConstraint(Compliance(negative_open,open >= 0,None,List(),None))          |Success          |                  |
|Negative Check|Error      |Success     |ComplianceConstraint(Compliance(negative_high,high >= 0,None,List(),None))          |Success          |                  |
|Negative Check|Error      |Success     |ComplianceConstraint(Compliance(negative_low,low >= 0,None,List(),None))            |Success          |                  |
|Negative Check|

## Test 4 - Determine Maximum Values ⚠️

For the fourth test, we want to find the maximum values in the dataset for the numerical fields. Extremum values can often be used to define an upper bound for the column values so we can define them as the threshold values. 

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Column Profiler Runner` to generate summary statistics for all the available columns. 
>2. Extract the maximum values for all the numeric columns in the data.
>
> *You may use as many cells as necessary*

In [211]:
#Maximum Values (Validity)
profiler_result = ColumnProfilerRunner(spark).onData(df.select([col for col in df.columns if col not in ["date", "stock"]])).run()

profiles = profiler_result.profiles

# Convert the dictionary to a list of Rows
profile_rows = []
for col_name, profile in profiles.items():
    row = Row(
        column=col_name,
        completeness=profile.completeness,
        data_type=profile.dataType,
        approx_distinct=profile.approximateNumDistinctValues,
        min_value=profile.minimum,
        max_value=profile.maximum,
        mean=profile.mean
    )
    profile_rows.append(row)

# Create a DataFrame from the list of Rows using the explicit schema
profile_df = spark.createDataFrame(profile_rows)
profile_df.show(truncate=False)

+---------+------------+----------+---------------+-------------------+-----------------+------------------+
|column   |completeness|data_type |approx_distinct|min_value          |max_value        |mean              |
+---------+------------+----------+---------------+-------------------+-----------------+------------------+
|open     |1.0         |Fractional|912            |0.0                |303.125          |1.1365598366658882|
|low      |1.0         |Fractional|1702           |0.06563635170459747|311.875          |18.943191553180316|
|close    |1.0         |Fractional|1793           |0.06607984006404877|313.75           |19.06198000288253 |
|volume   |1.0         |Fractional|1404           |0.0                |2.06928E7        |525223.0677290837 |
|adj_close|1.0         |Fractional|2547           |4.89296041905618E-7|148.7704620361328|7.208458180424869 |
|high     |1.0         |Fractional|1711           |0.06785380095243454|315.625          |19.17383405059962 |
+---------+--------

In [212]:
numeric_cols = [c for c, profile in result.profiles.items() if profile.dataType == 'Integral' or profile.dataType == 'Fractional']
max_values = {col: profile.maximum for col, profile in result.profiles.items() if col in numeric_cols}

print(max_values)

{'open': 303.125, 'low': 311.875, 'close': 313.75, 'volume': 20692800.0, 'adj_close': 148.7704620361328, 'high': 315.625}


## Test 5 - Stock Tickers 💹️

For the fifth test, we want to determine if the stock tickers contained in our dataset are consistent. To do this, you will need to make use of use of the metadata file to check that the stock names used in the dataframe are valid. 

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to determine if the stock tickers contained in the dataset appear in the metadata file.
>2. Display the results of your test.
>
> *You may use as many cells as necessary*

In [213]:
#Stocker Tickers (Validity)
# Extract ticker columns
parquet_tickers = df.select("stock").distinct()
metadata_tickers = metadata_df.select("NASDAQ Symbol").distinct()

# Create a check to verify if all tickers in the parquet dataset appear in the metadata dataset
check = Check(spark, CheckLevel.Error, "Ticker Verification") \
    .isContainedIn("stock", [row['NASDAQ Symbol'] for row in metadata_tickers.collect()])

# Run the verification suite
verification_result = VerificationSuite(spark) \
    .onData(parquet_tickers) \
    .addCheck(check) \
    .run()

# Show the result
result_df = VerificationResult.checkResultsAsDataFrame(spark, verification_result)
result_df.show()

+-------------------+-----------+------------+--------------------+-----------------+------------------+
|              check|check_level|check_status|          constraint|constraint_status|constraint_message|
+-------------------+-----------+------------+--------------------+-----------------+------------------+
|Ticker Verification|      Error|     Success|ComplianceConstra...|          Success|                  |
+-------------------+-----------+------------+--------------------+-----------------+------------------+



## Test 6 - Duplication 👥️
Lastly, we want to determine the uniqueness of the items found in the dataframe. You need to make use of the Verification Suite to check for the validity of the stock tickers. 

Similar to the previous notebook - `Data_profiling_student_version.ipynb`, the first thing to check will be if the primary key values within the dataset are unique - in our case, that will be a combination of the stock name and the date. Secondly, we want to check if the entries are all unique, which is done by checking for duplicates across that whole dataset.

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to determine the uniqueness of entries contained within the dataset.
>2. Display the results of your test.
>
> *You may use as many cells as necessary*



In [214]:
#Uniqueness
# Check for uniqueness based on the combination of `date` and `stock`
# Uniqueness check based on the combination of `date` and `stock`
check_1 = Check(spark, CheckLevel.Error, "Date and Stock Uniqueness Check") \
    .hasUniqueness(["date", "stock"], lambda x: x == 1)

# Uniqueness check for the entire rows
# Replace "field1", "field2", "field3" with the actual field names in your dataset
check_2 = Check(spark, CheckLevel.Error, "Row Uniqueness Check") \
    .hasUniqueness(df.columns, lambda x: x == 1)

# Run the verification suite for both checks
verification_result_1 = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(check_1) \
    .run()

verification_result_2 = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(check_2) \
    .run()

# Show the results
result_df_1 = VerificationResult.checkResultsAsDataFrame(spark, verification_result_1)
result_df_2 = VerificationResult.checkResultsAsDataFrame(spark, verification_result_2)

print("Uniqueness Check based on Date and Stock:")
result_df_1.show()

print("Uniqueness Check based on Entire Rows:")
result_df_2.show()


Uniqueness Check based on Date and Stock:
+--------------------+-----------+------------+--------------------+-----------------+------------------+
|               check|check_level|check_status|          constraint|constraint_status|constraint_message|
+--------------------+-----------+------------+--------------------+-----------------+------------------+
|Date and Stock Un...|      Error|     Success|UniquenessConstra...|          Success|                  |
+--------------------+-----------+------------+--------------------+-----------------+------------------+

Uniqueness Check based on Entire Rows:
+--------------------+-----------+------------+--------------------+-----------------+------------------+
|               check|check_level|check_status|          constraint|constraint_status|constraint_message|
+--------------------+-----------+------------+--------------------+-----------------+------------------+
|Row Uniqueness Check|      Error|     Success|UniquenessConstra...|  